In [ ]:
# Load necessary packages 
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2

In [ ]:
def select_slide(adata, s, batch_key="sample"):
    r"""This function selects the data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial experiments
    :param s: name of selected experiment
    :param batch_key: column in adata.obs listing experiment name for each location
    """

    slide = adata[adata.obs[batch_key].isin([s]), :].copy()
    s_keys = list(slide.uns["spatial"].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]

    slide.uns["spatial"] = {s_spatial: slide.uns["spatial"][s_spatial]}

    return slide

def prepare_adata(adata, cluster_key='spatial.cluster.3'):
    """
    Extracts relevant information from an AnnData object, prepares a DataFrame to help with pixel transformation.

    Parameters:
    - adata: AnnData object containing spatial transcriptomics data.
    - cluster_key: The key for cluster IDs in `adata.obs`.

    Returns:
    - df: DataFrame with extracted spatial data.
    """
    # Extract spot IDs, coordinates, and cluster IDs
    spot_ids = adata.obs_names  # Spot IDs (barcodes)
    pixel_coords = adata.obsm['spatial']  # Pixel coordinates (x, y)
    
    # Convert cluster IDs to numeric, handling any potential conversion issues
    cluster_ids = pd.to_numeric(adata.obs[cluster_key], errors='coerce')
    cluster_ids3 = pd.to_numeric(adata.obs['spatial.cluster.3'], errors='coerce')
    
    # Extract array row and column
    array_row = adata.obs['array_row']
    array_col = adata.obs['array_col']
    
    # Combine the data into a DataFrame
    df = pd.DataFrame({
        'sample': adata.obs['sample'],
        'spot_id': spot_ids,
        'x': pixel_coords[:, 0],
        'y': pixel_coords[:, 1],
        'x_original': pixel_coords[:, 0],
        'y_original': pixel_coords[:, 1],
        'cluster_id': cluster_ids.values,
        'cluster_id_3' : cluster_ids3.values,
        'array_row': array_row,
        'array_col': array_col
    })
    
    return df

def plot_spatial_data_with_keypoints(df, x_col='array_row', y_col='array_col', cluster_key='cluster_id', key_points=None, point_size=20, key_point_size=100):
    """
    Plots spatial data from a DataFrame with cluster coloring and highlights specified key points.
    
    Parameters:
    - df: DataFrame containing spatial data.
    - x_col: Column name for x coordinates.
    - y_col: Column name for y coordinates.
    - cluster_key: The key for cluster IDs in `df`.
    - key_points: List of tuples [(x, y), ...] representing points to highlight.
    - point_size: Size of the points in the main scatter plot.
    - key_point_size: Size of the highlighted key points.
    
    Returns:
    - key_points_xy: NumPy array of corresponding (x, y) coordinates for each key point.
    """
    
    # Define a custom color palette for clusters 1, 2, and 3
    palette = {1: 'blue', 2: 'orange', 3: 'green'}

    # Plot the main scatter plot with a contrasting palette
    plt.figure(figsize=(10, 10))
    sns.scatterplot(
        data=df,
        x=x_col,
        y=y_col,
        hue=cluster_key,
        palette=palette,  # High-contrast palette
        s=point_size,     # Adjust main point size
        edgecolor='w',
        legend=False
    )

    # Overlay each key point with a different color and size
    if key_points:
        key_points_xy = []
        for x_point, y_point in key_points:
            # Get the corresponding x and y coordinates
            key_point = df[(df[x_col] == x_point) & (df[y_col] == y_point)]
            if not key_point.empty:
                key_points_xy.append((key_point.iloc[0]['x'], key_point.iloc[0]['y']))
                sns.scatterplot(
                    data=key_point,
                    x=x_col,
                    y=y_col,
                    color='black',       # Color to make key points stand out
                    s=key_point_size,     # Adjust key point size for visibility
                    edgecolor='yellow'    # Edge color for additional contrast
                )

        # Convert list of key point coordinates to a NumPy array
        key_points_xy = np.array(key_points_xy, dtype=np.float32)
        return key_points_xy

    # Customize plot appearance
    plt.title("Spatial Data with Key Points Highlighted")
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    #plt.gca().invert_yaxis()  # Invert y-axis to match tissue orientation if needed
    plt.show()


def align_and_concatenate_spatial_data(df_src, df_dst, keypoints_src, keypoints_dst, cluster_key='cluster_id', palette=None):
    """
    Aligns spatial data from a source DataFrame to a destination DataFrame using an affine transformation,
    adds transformed coordinates to the source DataFrame, and concatenates the source and destination DataFrames.
    
    Parameters:
    - df_src: DataFrame containing source spatial data with 'x' and 'y' coordinates.
    - df_dst: DataFrame containing destination spatial data with 'x' and 'y' coordinates.
    - keypoints_src: NumPy array of selected source points [[x, y], ...] for transformation.
    - keypoints_dst: NumPy array of selected destination points [[x, y], ...] for transformation.
    - cluster_key: Column name for cluster IDs in `df_src` and `df_dst`.
    - palette: Optional dictionary to specify colors for each cluster ID.
    
    Returns:
    - df_combined: DataFrame containing both original and transformed source coordinates, and destination data.
    """
    
    # Step 1: Estimate the affine transformation matrix using selected key points
    M, inliers = cv2.estimateAffinePartial2D(keypoints_src, keypoints_dst, method=cv2.RANSAC)
    
    # Step 2: Apply the transformation matrix to the entire source DataFrame
    all_points_src = df_src[['x', 'y']].values.astype(np.float32)
    transformed_points = cv2.transform(np.array([all_points_src]), M)[0]
    
    # Step 3: Create new columns in df_src for transformed coordinates, explicitly rounded to two decimal places
    df_src['x_transformed'] = np.round(transformed_points[:, 0], 2)
    df_src['y_transformed'] = np.round(transformed_points[:, 1], 2)
    
    # Step 4: Fill NaN values in x_transformed and y_transformed with original x and y values
    df_dst['x_transformed'] = df_dst['x']
    df_dst['y_transformed'] = df_dst['y']
    
    # Step 5: Concatenate df_src with df_dst
    df_combined = pd.concat([df_src, df_dst], ignore_index=True)
    
    # Ensure that 'x' and 'y' in the combined dataframe are also rounded values
    df_combined['x'] = np.round(df_combined['x_transformed'], 2)
    df_combined['y'] = np.round(df_combined['y_transformed'], 2)
    
    # Drop the 'x_transformed' and 'y_transformed' columns
    df_combined = df_combined.drop(columns=['x_transformed', 'y_transformed'])
    
    # Step 6: Plot the source, destination, and transformed points for comparison
    plt.figure(figsize=(10, 10))
    sns.scatterplot(data=df_src, x='x', y='y', hue=cluster_key, palette=palette, s=1)
    sns.scatterplot(data=df_dst, x='x', y='y', hue=cluster_key, palette=palette, s=1)
    sns.scatterplot(data=df_src, x='x_transformed', y='y_transformed', hue=cluster_key, palette=palette, s=1)
    
    # Customize plot appearance
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("Alignment of Source Points to Destination")
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()
    
    return df_combined

def align_daughters_to_mother_jpg(df_src, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id', palette=None):
    """
    Aligns spatial data from a source DataFrame to a given .jpg image using an affine transformation,
    adjusts the coordinates in the source DataFrame, and plots the spots over the image.

    Parameters:
    - df_src: DataFrame containing source spatial data with 'x' and 'y' coordinates.
    - jpg_path: Path to the .jpg image to align the source data to.
    - keypoints_src: NumPy array of selected source points [[x, y], ...] for transformation.
    - keypoints_dst: NumPy array of selected destination points [[x, y], ...] for transformation.
    - cluster_key: Column name for cluster IDs in `df_src`.
    - palette: Optional dictionary to specify colors for each cluster ID.

    Returns:
    - df_src: DataFrame with transformed coordinates added as 'x_transformed' and 'y_transformed'.
    """
    # Load the image
    image = cv2.imread(jpg_path)
    if image is None:
        raise FileNotFoundError(f"Image at {jpg_path} not found.")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_height, img_width, _ = image.shape

    # Step 1: Estimate the affine transformation matrix using selected key points
    M, inliers = cv2.estimateAffinePartial2D(keypoints_src, keypoints_dst, method=cv2.RANSAC)

    # Step 2: Apply the transformation matrix to the entire source DataFrame
    all_points_src = df_src[['x', 'y']].values.astype(np.float32)
    transformed_points = cv2.transform(np.array([all_points_src]), M)[0]

    # Step 3: Add transformed coordinates to df_src
    df_src['x_transformed'] = np.round(transformed_points[:, 0], 2)
    df_src['y_transformed'] = np.round(transformed_points[:, 1], 2)

    # Step 4: Plot the transformed points on the image
    plt.figure(figsize=(10, 10))
    plt.imshow(image, extent=[0, img_width, img_height, 0])  # Adjust extents for correct orientation
    plt.gca().invert_yaxis()  # Match image coordinate system

    if cluster_key in df_src.columns and palette:
        for cluster, color in palette.items():
            cluster_points = df_src[df_src[cluster_key] == cluster]
            plt.scatter(cluster_points['x_transformed'], cluster_points['y_transformed'], 
                        c=color, label=str(cluster), s=2)
    else:
        plt.scatter(df_src['x_transformed'], df_src['y_transformed'], c='red', s=2)

    # Step 5: Customize the plot
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("Aligned Spatial Data on Image")
    plt.legend(loc='upper right', markerscale=5)
    plt.show()

    return df_src


def plot_keypoints_on_image(jpg_path, keypoints):
    """
    Reads a .jpg image and overlays keypoints on it.

    Parameters:
    - jpg_path: Path to the .jpg image.
    - keypoints: NumPy array of keypoints [[x, y], ...] to plot on the image.

    Returns:
    - None: Displays the plot.
    """
    # Load the image
    image = cv2.imread(jpg_path)
    if image is None:
        raise FileNotFoundError(f"Image at {jpg_path} not found.")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Plot the image
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    
    # Overlay keypoints
    keypoints = np.array(keypoints)
    plt.scatter(keypoints[:, 0], keypoints[:, 1], c='black', s=20, label='Keypoints')

    # Add plot details
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("Keypoints on Image")
    plt.legend(loc='upper right')
    plt.gca().invert_yaxis()  # Match image coordinate system if needed
    plt.show()
    
    
    


def plot_coordinates_on_image(image, df, cluster_key=None, palette=None, s = 2):
    """
    Plot coordinates from a dataframe on top of a given image.

    Parameters:
    - image: np.array
        The image on which the coordinates will be plotted (in RGB format).
    - df: pd.DataFrame
        DataFrame containing coordinates to be plotted. Must have 'x' and 'y' columns.
    - cluster_key: str, optional
        Column in the DataFrame representing clusters for coloring (default is None).
    - palette: dict, optional
        Dictionary mapping cluster labels to colors (default is None).
    
    Returns:
    - None
    """
    if 'x' not in df.columns or 'y' not in df.columns:
        raise ValueError("DataFrame must contain 'x' and 'y' columns for coordinates.")

    # Get image dimensions for plotting extents
    img_height, img_width, _ = image.shape

    # Create the plot
    plt.figure(figsize=(10, 10))
    plt.imshow(image, extent=[0, img_width, img_height, 0])  # Adjust extents for correct orientation
    plt.gca().invert_yaxis()  # Match image coordinate system
    
    if cluster_key in df.columns and palette:
        for cluster, color in palette.items():
            cluster_points = df[df[cluster_key] == cluster]
            plt.scatter(cluster_points['x_transformed'], cluster_points['y_transformed'], 
                        c=color, label=str(cluster), s=s)
    else:
        plt.scatter(df['x_transformed'], df['y_transformed'], c='red', s=s)

    

    # Customize the plot
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("Spatial Data on Image")
    if cluster_key and palette:
        plt.legend(loc='upper right', markerscale=5)
    plt.show()



# Load daugther data

In [ ]:
# Load the data
## this is an adata object that was had generated which combined all the daughter caputres into one large obj
#load the original merged 10x visium processed data
adata = sc.read_h5ad("Human_ICL_bayespace_13_clusters.h5ad")

# Align 612-1 to mother
## Identify mother key points

In [ ]:
# plot the mother keypoints
# Define keypoints
keypoints = np.array([
    [3456.0000, 2368.0000],
    [3807.5000, 3247.5000],
    #[4063.5000, 4063.5000],
    #[4223.5000, 4911.5000],
    #[4399.5000, 5807.5000],
    #[6047.5000, 4415.5000],
    [7791.5000, 6895.5000],
    [5215.5000, 2143.5000],
    #[5871.5000, 3071.5000],
    #[6575.5000, 3887.5000],
    #[7023.5000, 4479.5000],
    [7535.5000, 5007.5000]#,
    #[6383.5000, 5759.5000]#,
    #[5231.5000, 5407.5000]
])

# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

## Identify 612-1 daughter key points

In [ ]:
# Format the daughter data
# Separate the data to the specific visium daughter captures we want to align
sample_612_1 = select_slide(adata, "612-1")
sample_612_1 = prepare_adata(sample_612_1)

In [ ]:
plot_spatial_data_with_keypoints(sample_612_1, point_size=40)

In [ ]:
# Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(sample_612_1,key_points = [(22,32), (25,49),(65,127), (41,31), (64, 90)], point_size=40)

In [ ]:
# Align 612-1 to mother image 
# Example usage
jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[2974., -200.],
       [3036., -219.],
       [3323., -474.],
       [2970., -322.],
       [3187., -468.]])  # Source points
keypoints_dst = np.array([
    [3456.0000, 2368.0000],
    [3807.5000, 3247.5000],
    #[4063.5000, 4063.5000],
    #[4223.5000, 4911.5000],
    #[4399.5000, 5807.5000],
    #[6047.5000, 4415.5000],
    [7791.5000, 6895.5000],
    [5215.5000, 2143.5000],
    #[5871.5000, 3071.5000],
    #[6575.5000, 3887.5000],
    #[7023.5000, 4479.5000],
    [7535.5000, 5007.5000]#,
    #[6383.5000, 5759.5000]#,
    #[5231.5000, 5407.5000]
])
palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_612_1 = align_daughters_to_mother_jpg(sample_612_1, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

In [ ]:
df_src_transformed_612_1

# Align 613-3 to mother 
## Identify mother key points

In [ ]:
# plot the mother keypoints
# Define keypoints
keypoints = np.array([
    [4256.0000, 5706.0000],
    #[4495.5000, 6569.5000],
    #[4831.5000, 7353.5000],
    #[5071.5000, 8073.5000],
    [5231.5000, 9081.5000],
    #[5135.5000, 9945.5000],
    #[4687.5000, 10617.5000],
    #[4159.5000, 11177.5000],
    [9007.5000, 9129.5000],
    #[9471.5000, 9593.5000],
    #[10223.5000, 7497.5000],
    #[10319.5000, 6937.5000],
    [9839.5000, 6617.5000],
    #[9215.5000, 6249.5000],
    [8575.5000, 5817.5000]#,
    #[7887.5000, 5225.5000],
    #[7247.5000, 4665.5000],
    #[6879.5000, 4281.5000]
])


# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

## Identify 613-3 Daughter key points 

In [ ]:
# Format the daughter data
# Separate the data to the specific visium daughter captures we want to align
sample_613_3 = select_slide(adata, "613-3")
sample_613_3 = prepare_adata(sample_613_3)

In [ ]:
plot_spatial_data_with_keypoints(sample_613_3, point_size= 40)

In [ ]:
plot_spatial_data_with_keypoints(sample_613_3,key_points = [(15, 109),(48,90),(46,24),(21,13),(14,36)], point_size= 40)

In [ ]:
# Align 613-3 to mother image 
# Example usage
jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[7468., -146.],
       [7401., -351.],
       [7165., -339.],
       [7126., -183.],
       [7208., -140.]])  # Source points
keypoints_dst = np.array([
    [4256.0000, 5706.0000],
    #[4495.5000, 6569.5000],
    #[4831.5000, 7353.5000],
    #[5071.5000, 8073.5000],
    [5231.5000, 9081.5000],
    #[5135.5000, 9945.5000],
    #[4687.5000, 10617.5000],
    #[4159.5000, 11177.5000],
    [9007.5000, 9129.5000],
    #[9471.5000, 9593.5000],
    #[10223.5000, 7497.5000],
    #[10319.5000, 6937.5000],
    [9839.5000, 6617.5000],
    #[9215.5000, 6249.5000],
    [8575.5000, 5817.5000]#,
    #[7887.5000, 5225.5000],
    #[7247.5000, 4665.5000],
    #[6879.5000, 4281.5000]
])
palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_613_3 = align_daughters_to_mother_jpg(sample_613_3, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

# Combined Aligned Results

In [ ]:
combined_results = pd.concat([df_src_transformed_613_3, df_src_transformed_612_1])

In [ ]:
combined_results

In [ ]:
image =  cv2.imread(jpg_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plot_coordinates_on_image(image, combined_results, cluster_key='cluster_id',palette=palette)

# Align 613-2 to the mother

In [ ]:
# plot the mother keypoints
# Define keypoints
keypoints = np.array([
    [10280.0000, 6972.0000],
    [11143.5000, 7467.5000],
    [11767.5000, 7819.5000],
    [12295.5000, 8059.5000],
    [12935.5000, 8283.5000],
    [13559.5000, 8475.5000],
    [14247.5000, 8571.5000],
    [14791.5000, 8635.5000],
    [9447.5000, 7563.5000],
    [10183.5000, 7531.5000],
    [10839.5000, 7819.5000],
    [11623.5000, 8139.5000],
    [9415.5000, 9531.5000],
    [9991.5000, 9979.5000],
    [10583.5000, 10331.5000],
    [11063.5000, 10635.5000],
    [11511.5000, 10987.5000],
    [12087.5000, 11515.5000],
    [12519.5000, 12059.5000],
    [14551.5000, 9051.5000]
])
# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
# plot the mother keypoints
# Define keypoints
keypoints = np.array([
    [10280.0000, 6972.0000],
    #[11143.5000, 7467.5000],
    [11767.5000, 7819.5000],
    #[12295.5000, 8059.5000],
    #[12935.5000, 8283.5000],
    #[13559.5000, 8475.5000],
    #[14247.5000, 8571.5000],
    [14791.5000, 8635.5000],
    [9447.5000, 7563.5000],
    #[10183.5000, 7531.5000],
    #[10839.5000, 7819.5000],
    #[11623.5000, 8139.5000],
    #[9415.5000, 9531.5000],
    [9991.5000, 9979.5000]#,
    #[10583.5000, 10331.5000],
    #[11063.5000, 10635.5000],
    #[11511.5000, 10987.5000],
    #[12087.5000, 11515.5000],
    #[12519.5000, 12059.5000]#,
    #[14551.5000, 9051.5000]
])
# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

## Identify 613-2 daughter key points 

In [ ]:
# Format the daughter data
# Separate the data to the specific visium daughter captures we want to align
sample_613_2 = select_slide(adata, "613-2")
sample_613_2 = prepare_adata(sample_613_2)

In [ ]:
plot_spatial_data_with_keypoints(sample_613_2,  point_size = 40)

In [ ]:
# Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(sample_613_2, key_points = [(13,39) ,(27,51),(55,59),(6,50),(13,87)], point_size = 40)

In [ ]:
# Align 613-2 to mother image 
# Example usage
jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[6513., -136.],
       [6556., -222.],
       [6585., -396.],
       [6552.,  -92.],
       [6684., -135.]])  # Source points
keypoints_dst = np.array([
    [10280.0000, 6972.0000],
    #[11143.5000, 7467.5000],
    [11767.5000, 7819.5000],
    #[12295.5000, 8059.5000],
    #[12935.5000, 8283.5000],
    #[13559.5000, 8475.5000],
    #[14247.5000, 8571.5000],
    [14791.5000, 8635.5000],
    [9447.5000, 7563.5000],
    #[10183.5000, 7531.5000],
    #[10839.5000, 7819.5000],
    #[11623.5000, 8139.5000],
    #[9415.5000, 9531.5000],
    [9991.5000, 9979.5000]#,
    #[10583.5000, 10331.5000],
    #[11063.5000, 10635.5000],
    #[11511.5000, 10987.5000],
    #[12087.5000, 11515.5000],
    #[12519.5000, 12059.5000]#,
    #[14551.5000, 9051.5000]
])
palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_613_2 = align_daughters_to_mother_jpg(sample_613_2, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

# Combined the results 

In [ ]:
combined_results2 = pd.concat([combined_results, df_src_transformed_613_2])

In [ ]:
plot_coordinates_on_image(image, combined_results2, cluster_key='cluster_id',palette=palette)

# Align 612-2 to mother

In [ ]:
# plot the mother keypoints
# Define keypoints
keypoints = np.array([
    [5106.0000, 10002.0000],
    #[4889.5000, 10385.5000],
    #[4637.5000, 10745.5000],
    #[4337.5000, 11081.5000],
    #[4085.5000, 11333.5000],
    #[3749.5000, 11537.5000],
    [4349.5000, 13697.5000],
    [4709.5000, 13361.5000],
    #[5069.5000, 13025.5000],
    [5489.5000, 12737.5000],
    #[5885.5000, 12449.5000],
    #[6389.5000, 12305.5000],
    #[7013.5000, 12293.5000],
    #[7673.5000, 12425.5000],
    #[8333.5000, 12797.5000],
    #[8945.5000, 13217.5000],
    #[9783.5000, 16119.5000],
    #[9335.5000, 16199.5000],
    [8839.5000, 16295.5000]#,
    #[8391.5000, 16503.5000]
])


# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
# Daughter key points 

In [ ]:
sample_612_2 = select_slide(adata, "612-2")
# Preapre into a workable dataframe
sample_612_2 = prepare_adata(sample_612_2)

In [ ]:
plot_spatial_data_with_keypoints(sample_612_2, point_size=40)

In [ ]:
plot_spatial_data_with_keypoints(sample_612_2,key_points = [(5,105), (44,116),(41,109),(34,96),(69,31)], point_size=40)

In [ ]:
# Align 612-2 to mother image 
# Example usage
jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[3935.,  -88.],
       [3976., -334.],
       [3950., -315.],
       [3903., -271.],
       [3668., -493.]])  # Source points
keypoints_dst = np.array([
    [5106.0000, 10002.0000],
    #[4889.5000, 10385.5000],
    #[4637.5000, 10745.5000],
    #[4337.5000, 11081.5000],
    #[4085.5000, 11333.5000],
    #[3749.5000, 11537.5000],
    [4349.5000, 13697.5000],
    [4709.5000, 13361.5000],
    #[5069.5000, 13025.5000],
    [5489.5000, 12737.5000],
    #[5885.5000, 12449.5000],
    #[6389.5000, 12305.5000],
    #[7013.5000, 12293.5000],
    #[7673.5000, 12425.5000],
    #[8333.5000, 12797.5000],
    #[8945.5000, 13217.5000],
    #[9783.5000, 16119.5000],
    #[9335.5000, 16199.5000],
    [8839.5000, 16295.5000]#,
    #[8391.5000, 16503.5000]
])

palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_612_2 = align_daughters_to_mother_jpg(sample_612_2, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

In [ ]:
combined_results3 = pd.concat([combined_results2, df_src_transformed_612_2])

In [ ]:
plot_coordinates_on_image(image, combined_results3, cluster_key='cluster_id',palette=palette)

# Align 611-2

In [ ]:
keypoints = np.array([
    [288.0000, 14838.0000],
    #[671.5000, 14453.5000],
    [1055.5000, 14117.5000],
    #[1391.5000, 13829.5000],
    #[1679.5000, 13469.5000],
    #[1991.5000, 13157.5000],
    [2399.5000, 12773.5000],
    #[2711.5000, 12437.5000],
    #[887.5000, 18389.5000],
    #[887.5000, 17813.5000],
    #[1103.5000, 17261.5000],
    #[1391.5000, 16733.5000],
    #[1799.5000, 16325.5000],
    #[2231.5000, 15893.5000],
    [2663.5000, 15461.5000]#,
   #[3095.5000, 15005.5000],
    #[3647.5000, 14549.5000],
    #[4031.5000, 14165.5000],
    #[4391.5000, 13733.5000]
])



# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
sample_611_2 = select_slide(adata, "611-2")
# Preapre into a workable dataframe
sample_611_2 = prepare_adata(sample_611_2)

In [ ]:
plot_spatial_data_with_keypoints(sample_611_2,  point_size = 40)

In [ ]:
#Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(sample_611_2, key_points = [(44,84),(36,70),(22,46),(50,40)], point_size = 40)

In [ ]:
# Align 611-2 to mother image 

jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[1066., -337.],
       [1015., -287.],
       [ 927., -198.],
       [ 906., -376.]])  # Source points
keypoints_dst = np.array([
    [288.0000, 14838.0000],
    #[671.5000, 14453.5000],
    [1055.5000, 14117.5000],
    #[1391.5000, 13829.5000],
    #[1679.5000, 13469.5000],
    #[1991.5000, 13157.5000],
    [2399.5000, 12773.5000],
    #[2711.5000, 12437.5000],
    #[887.5000, 18389.5000],
    #[887.5000, 17813.5000],
    #[1103.5000, 17261.5000],
    #[1391.5000, 16733.5000],
    #[1799.5000, 16325.5000],
    #[2231.5000, 15893.5000],
    [2663.5000, 15461.5000]#,
   #[3095.5000, 15005.5000],
    #[3647.5000, 14549.5000],
    #[4031.5000, 14165.5000],
    #[4391.5000, 13733.5000]
])
palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_611_2 = align_daughters_to_mother_jpg(sample_611_2, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

In [ ]:
# Combined the results 
combined_results4= pd.concat([combined_results3, df_src_transformed_611_2])

In [ ]:
plot_coordinates_on_image(image, combined_results4, cluster_key='cluster_id',palette=palette)

## Addition of 611-1

In [ ]:
keypoints = np.array([
    [288.0000, 14838.0000],
    [671.5000, 14453.5000],
    [1055.5000, 14117.5000],
    [1391.5000, 13829.5000],
    [1679.5000, 13469.5000],
    [1991.5000, 13157.5000],
    [2399.5000, 12773.5000],
    [2711.5000, 12437.5000],
    [887.5000, 18389.5000],
    [887.5000, 17813.5000],
    [1103.5000, 17261.5000],
    [1391.5000, 16733.5000],
    [1799.5000, 16325.5000],
    [2231.5000, 15893.5000],
    [2663.5000, 15461.5000],
    [3095.5000, 15005.5000],
    [3647.5000, 14549.5000],
    [4031.5000, 14165.5000],
    [4391.5000, 13733.5000]
])


# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
sample_611_1 = select_slide(adata, "611-1")
# Preapre into a workable dataframe
sample_611_1 = prepare_adata(sample_611_1)

In [ ]:
plot_spatial_data_with_keypoints(sample_611_1, point_size = 40)

In [ ]:
#Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(sample_611_1, key_points = [(69,59),(77,69),(26,60),(48,80)], point_size = 40)

In [ ]:
# Align 611-2 to mother image 

jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[ 273., -494.],
       [ 310., -544.],
       [ 275., -220.],
       [ 349., -360.]])  # Source points
keypoints_dst = np.array([
    #[288.0000, 14838.0000],
    #[671.5000, 14453.5000],
    #[1055.5000, 14117.5000],
    [1391.5000, 13829.5000],
    #[1679.5000, 13469.5000],
    [1991.5000, 13157.5000],
    #[2399.5000, 12773.5000],
    #[2711.5000, 12437.5000],
    #[887.5000, 18389.5000],
    [887.5000, 17813.5000],
    #[1103.5000, 17261.5000],
    #[1391.5000, 16733.5000],
    #[1799.5000, 16325.5000],
    [2231.5000, 15893.5000]#,
    #[2663.5000, 15461.5000],
    #[3095.5000, 15005.5000],
    #[3647.5000, 14549.5000],
    #[4031.5000, 14165.5000],
    #[4391.5000, 13733.5000]
])
palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_611_1 = align_daughters_to_mother_jpg(sample_611_1, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

In [ ]:
# Combined the results 
combined_results5= pd.concat([combined_results4, df_src_transformed_611_1])

In [ ]:
plot_coordinates_on_image(image, combined_results5, cluster_key='cluster_id',palette=palette)

# Addition of Sample 695-1

In [ ]:
keypoints = np.array([
    #[15465.0000, 17463.0000],
    #[15404.5000, 17666.5000],
    #[15320.5000, 18227.5000],
    #[15308.5000, 18623.5000],
    #[15308.5000, 19067.5000],
    #[15380.5000, 19415.5000],
    #[15164.5000, 20099.5000],
    #[15008.5000, 20651.5000],
    [15488.5000, 20807.5000],
    #[16076.5000, 20699.5000],
    #[16496.5000, 20471.5000],
    #[16796.5000, 20126.5000],
    [16868.5000, 19874.5000],
    #[17000.5000, 19178.5000],
    #[17396.5000, 18857.5000],
    #[17792.5000, 18497.5000],
    #[18020.5000, 18113.5000],
    #[18200.5000, 17729.5000],
    #[18488.5000, 17333.5000],
    #[17264.5000, 21365.5000],
    [17480.5000, 20933.5000],
    #[17756.5000, 20417.5000],
    #[17900.5000, 20141.5000],
    #[18056.5000, 19805.5000],
    #[18212.5000, 19553.5000],
    #[18428.5000, 19301.5000],
    #[18632.5000, 19049.5000],
    #[18872.5000, 18845.5000],
    #[19076.5000, 18653.5000],
    #[19280.5000, 18461.5000],
    [19568.5000, 18317.5000]#,
    #[19904.5000, 18185.5000],
    #[20264.5000, 18149.5000],
    #[20708.5000, 18293.5000],
    #[21044.5000, 18461.5000],
    #[21380.5000, 18761.5000]
])



# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
sample_695_1 = select_slide(adata, "695-1")
# Preapre into a workable dataframe
sample_695_1 = prepare_adata(sample_695_1)

In [ ]:
plot_spatial_data_with_keypoints(sample_695_1,  point_size = 40)

In [ ]:
#Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(sample_695_1, key_points = [(28,120), (40,100),(48,116),(64,66)], point_size = 40)

In [ ]:
jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[ 1907., -4429.],
       [ 1836., -4504.],
       [ 1894., -4554.],
       [ 1715., -4653.]])  # Source points
keypoints_dst = np.array([
    #[15465.0000, 17463.0000],
    #[15404.5000, 17666.5000],
    #[15320.5000, 18227.5000],
    #[15308.5000, 18623.5000],
    #[15308.5000, 19067.5000],
    #[15380.5000, 19415.5000],
    #[15164.5000, 20099.5000],
    #[15008.5000, 20651.5000],
    [15488.5000, 20807.5000],
    #[16076.5000, 20699.5000],
    #[16496.5000, 20471.5000],
    #[16796.5000, 20126.5000],
    [16868.5000, 19874.5000],
    #[17000.5000, 19178.5000],
    #[17396.5000, 18857.5000],
    #[17792.5000, 18497.5000],
    #[18020.5000, 18113.5000],
    #[18200.5000, 17729.5000],
    #[18488.5000, 17333.5000],
    #[17264.5000, 21365.5000],
    [17480.5000, 20933.5000],
    #[17756.5000, 20417.5000],
    #[17900.5000, 20141.5000],
    #[18056.5000, 19805.5000],
    #[18212.5000, 19553.5000],
    #[18428.5000, 19301.5000],
    #[18632.5000, 19049.5000],
    #[18872.5000, 18845.5000],
    #[19076.5000, 18653.5000],
    #[19280.5000, 18461.5000],
    [19568.5000, 18317.5000]#,
    #[19904.5000, 18185.5000],
    #[20264.5000, 18149.5000],
    #[20708.5000, 18293.5000],
    #[21044.5000, 18461.5000],
    #[21380.5000, 18761.5000]
])

palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_695_1 = align_daughters_to_mother_jpg(sample_695_1, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

In [ ]:
# Combined the results 
combined_results6= pd.concat([combined_results5, df_src_transformed_695_1])
plot_coordinates_on_image(image, combined_results6, cluster_key='cluster_id',palette=palette)

# Addition of sample 696-2

In [ ]:
keypoints = np.array([
    #[15933.0000, 20742.0000],
    #[16304.5000, 20537.5000],
    #[16724.5000, 20225.5000],
    #[16868.5000, 19769.5000],
    #[17120.5000, 19241.5000],
    #[17672.5000, 18785.5000],
    [17912.5000, 18305.5000],
    [18152.5000, 17861.5000],
    #[18560.5000, 17369.5000],
    [18908.5000, 17045.5000],
    #[19448.5000, 16721.5000],
    #[20024.5000, 16349.5000],
    #[20444.5000, 16085.5000],
    #[20900.5000, 15761.5000],
    #[19724.5000, 18149.5000],
    #[20180.5000, 18113.5000],
    #[20684.5000, 18233.5000],
    #[21140.5000, 18449.5000],
    [21524.5000, 18773.5000]#,
    #[21860.5000, 19229.5000]
])


# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
sample_696_2 = select_slide(adata, "696-2")
# Preapre into a workable dataframe
sample_696_2 = prepare_adata(sample_696_2)

In [ ]:
plot_spatial_data_with_keypoints(sample_696_2, point_size = 40)

In [ ]:
#Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(sample_696_2, key_points = [(70,110),(65,107),(55,97),(67,45)], point_size = 40)

In [ ]:
jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[ 5370., -4687.],
       [ 5359., -4656.],
       [ 5324., -4594.],
       [ 5138., -4669.]])  # Source points
keypoints_dst = np.array([
    #[15933.0000, 20742.0000],
    #[16304.5000, 20537.5000],
    #[16724.5000, 20225.5000],
    #[16868.5000, 19769.5000],
    #[17120.5000, 19241.5000],
    #[17672.5000, 18785.5000],
    [17912.5000, 18305.5000],
    [18152.5000, 17861.5000],
    #[18560.5000, 17369.5000],
    [18908.5000, 17045.5000],
    #[19448.5000, 16721.5000],
    #[20024.5000, 16349.5000],
    #[20444.5000, 16085.5000],
    #[20900.5000, 15761.5000],
    #[19724.5000, 18149.5000],
    #[20180.5000, 18113.5000],
    #[20684.5000, 18233.5000],
    #[21140.5000, 18449.5000],
    [21524.5000, 18773.5000]#,
    #[21860.5000, 19229.5000]
])


palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_696_2 = align_daughters_to_mother_jpg(sample_696_2, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

In [ ]:
# Combined the results 
combined_results7= pd.concat([combined_results6, df_src_transformed_696_2])
plot_coordinates_on_image(image, combined_results7, cluster_key='cluster_id',palette=palette)

# Addition of 695-4

In [ ]:
keypoints = np.array([
    [20331.0000, 528.0000],
    [20378.5000, 1135.5000],
    [20522.5000, 1823.5000],
    [20506.5000, 2463.5000],
    [20474.5000, 3071.5000],
    [20410.5000, 3519.5000],
    [20346.5000, 4111.5000],
    [20234.5000, 4591.5000],
    [20106.5000, 5167.5000],
    [22058.5000, 159.5000],
    [22074.5000, 767.5000],
    [22122.5000, 1311.5000],
    [22138.5000, 1807.5000],
    [22138.5000, 2239.5000],
    [22138.5000, 2671.5000],
    [22202.5000, 3103.5000],
    [22234.5000, 3551.5000],
    [22378.5000, 3967.5000],
    [22522.5000, 4335.5000],
    [22826.5000, 4703.5000],
    [23226.5000, 4863.5000],
    [23530.5000, 4991.5000],
    [23930.5000, 5007.5000],
    [24314.5000, 4927.5000],
    [19978.5000, 5551.5000],
    [19786.5000, 5823.5000],
    [20314.5000, 6591.5000],
    [20650.5000, 6303.5000],
    [21114.5000, 6079.5000],
    [21514.5000, 6367.5000]
])




# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
sample_695_4 = select_slide(adata, "695-4")
# Preapre into a workable dataframe
sample_695_4 = prepare_adata(sample_695_4)

In [ ]:
plot_spatial_data_with_keypoints(sample_695_4,  point_size = 40)

In [ ]:
#Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(sample_695_4, key_points = [(22,36),(18,34),(46,70),(8,40),(13,53)], point_size = 40)

In [ ]:
keypoints = np.array([
    #[20331.0000, 528.0000],
    #[20378.5000, 1135.5000],
    #[20522.5000, 1823.5000],
    #[20506.5000, 2463.5000],
    #[20474.5000, 3071.5000],
    #[20410.5000, 3519.5000],
    #[20346.5000, 4111.5000],
    #[20234.5000, 4591.5000],
    [20106.5000, 5167.5000],
    #[22058.5000, 159.5000],
    #[22074.5000, 767.5000],
    #[22122.5000, 1311.5000],
    #[22138.5000, 1807.5000],
    #[22138.5000, 2239.5000],
    [22138.5000, 2671.5000],
    #[22202.5000, 3103.5000],
    #[22234.5000, 3551.5000],
    #[22378.5000, 3967.5000],
    #[22522.5000, 4335.5000],
    #[22826.5000, 4703.5000],
    #[23226.5000, 4863.5000],
    #[23530.5000, 4991.5000],
    #[23930.5000, 5007.5000],
    #[24314.5000, 4927.5000],
    [19978.5000, 5551.5000],
    #[19786.5000, 5823.5000],
    [20314.5000, 6591.5000],
    #[20650.5000, 6303.5000],
    [21114.5000, 6079.5000]#,
    #[21514.5000, 6367.5000]
])




# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[ 3709., -4394.],
       [ 3702., -4369.],
       [ 3831., -4542.],
       [ 3723., -4307.],
       [ 3770., -4338.]])  # Source points
keypoints_dst = np.array([
    #[20331.0000, 528.0000],
    #[20378.5000, 1135.5000],
    #[20522.5000, 1823.5000],
    #[20506.5000, 2463.5000],
    #[20474.5000, 3071.5000],
    #[20410.5000, 3519.5000],
    #[20346.5000, 4111.5000],
    #[20234.5000, 4591.5000],
    [20106.5000, 5167.5000],
    #[22058.5000, 159.5000],
    #[22074.5000, 767.5000],
    #[22122.5000, 1311.5000],
    #[22138.5000, 1807.5000],
    #[22138.5000, 2239.5000],
    [22138.5000, 2671.5000],
    #[22202.5000, 3103.5000],
    #[22234.5000, 3551.5000],
    #[22378.5000, 3967.5000],
    #[22522.5000, 4335.5000],
    #[22826.5000, 4703.5000],
    #[23226.5000, 4863.5000],
    #[23530.5000, 4991.5000],
    #[23930.5000, 5007.5000],
    #[24314.5000, 4927.5000],
    [19978.5000, 5551.5000],
    #[19786.5000, 5823.5000],
    [20314.5000, 6591.5000],
    #[20650.5000, 6303.5000],
    [21114.5000, 6079.5000]#,
    #[21514.5000, 6367.5000]
])




palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_694_4 = align_daughters_to_mother_jpg(sample_695_4, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

In [ ]:
# Combined the results 
combined_results8= pd.concat([combined_results7, df_src_transformed_694_4])
plot_coordinates_on_image(image, combined_results8, cluster_key='cluster_id',palette=palette)

# Addition of 612-4

In [ ]:
keypoints = np.array([
    #[9912.0000, 16050.0000],
    [9407.5000, 16073.5000],
    #[8879.5000, 16034.5000],
    [8543.5000, 16409.5000],
    #[8039.5000, 17105.5000],
    #[7655.5000, 17825.5000],
    #[7223.5000, 18659.5000],
    [7103.5000, 19691.5000],
    [7487.5000, 20747.5000]#,
    #[8135.5000, 21467.5000],
    #[9143.5000, 21779.5000]
])


# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
sample_612_4 = select_slide(adata, "612-4")
# Preapre into a workable dataframe
sample_612_4 = prepare_adata(sample_612_4)

In [ ]:
plot_spatial_data_with_keypoints(sample_612_4, point_size = 40)

In [ ]:
#Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(sample_612_4, key_points = [(61,107),(61,91),(34,44),(22,44)], point_size = 40)

In [ ]:
jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[5350., -446.],
       [5292., -446.],
       [5119., -274.],
       [5119., -198.]])  # Source points
keypoints_dst = np.array([
    #[9912.0000, 16050.0000],
    [9407.5000, 16073.5000],
    #[8879.5000, 16034.5000],
    [8543.5000, 16409.5000],
    #[8039.5000, 17105.5000],
    #[7655.5000, 17825.5000],
    #[7223.5000, 18659.5000],
    [7103.5000, 19691.5000],
    [7487.5000, 20747.5000]#,
    #[8135.5000, 21467.5000],
    #[9143.5000, 21779.5000]
])



palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_612_4 = align_daughters_to_mother_jpg(sample_612_4, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

In [ ]:
# Combined the results 
combined_results9= pd.concat([combined_results8, df_src_transformed_612_4])
plot_coordinates_on_image(image, combined_results9, cluster_key='cluster_id',palette=palette)

In [ ]:
# Combined the results 
combined_results9

In [ ]:
import csv
combined_results9.to_csv("combined_transformed_locs.tsv", sep="\t", index=False, quoting=csv.QUOTE_NONE)

# Align daugther 696-1

In [ ]:
# plot the mother keypoints
# Define keypoints
keypoints = np.array([
    [19232.0000, 7744.0000],
    [19615.5000, 7359.5000],
    [19999.5000, 6943.5000],
    [20447.5000, 6559.5000],
    [20991.5000, 6239.5000],
    [21471.5000, 6367.5000],
    [21663.5000, 7007.5000],
    [21791.5000, 7679.5000],
    [22079.5000, 8095.5000],
    [22879.5000, 8191.5000],
    [22943.5000, 8799.5000],
    [22783.5000, 9503.5000],
    [22591.5000, 10079.5000],
    [22559.5000, 10655.5000]
])

# Path to the image
jpg_path =  "he-scaled.jpg"

# Plot keypoints on the image
plot_keypoints_on_image(jpg_path, keypoints)

In [ ]:
sample_696_1 = select_slide(adata, "696-1")
# Preapre into a workable dataframe
sample_696_1 = prepare_adata(sample_696_1)

In [ ]:
plot_spatial_data_with_keypoints(sample_696_1, point_size = 40)

In [ ]:
#Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(sample_696_1, key_points = [(24,20),(50,44),(61,59),(56,100)], point_size = 40)

In [ ]:
jpg_path = "he-scaled.jpg"
keypoints_src = np.array([[ 4340., -4407.],
       [ 4426., -4568.],
       [ 4480., -4636.],
       [ 4619., -4605.]])  # Source points
keypoints_dst = np.array([
    #[19232.0000, 7744.0000],
    #[19615.5000, 7359.5000],
    [19999.5000, 6943.5000],
    #[20447.5000, 6559.5000],
    #[20991.5000, 6239.5000],
    #[21471.5000, 6367.5000],
    #[21663.5000, 7007.5000],
    #[21791.5000, 7679.5000],
    [22079.5000, 8095.5000],
    #[22879.5000, 8191.5000],
    [22943.5000, 8799.5000],
    #[22783.5000, 9503.5000],
    #[22591.5000, 10079.5000],
    [22559.5000, 10655.5000]
])


palette = {1: 'blue', 2: 'orange', 3: 'green'}
# Align and plot
df_src_transformed_696_1 = align_daughters_to_mother_jpg(sample_696_1, jpg_path, keypoints_src, keypoints_dst, cluster_key='cluster_id',palette=palette
)

In [ ]:
#Identify the key points manaully based on the image alignment using the row and columns 
plot_spatial_data_with_keypoints(combined_results ,x_col='x_transformed', y_col='y_transformed', point_size = 2)